In [4]:
import pickle
# import omegaconf
import warnings
import pickle
import numpy as np
warnings.filterwarnings("ignore", category=DeprecationWarning)

from load_results import load_result_dataset
import pandas as pd
pn1 = 'linearprobe_50epochs_edge_paper_final2'
pn2 = 'linearprobe_50epochs_paper_final2'

final_data = load_result_dataset(pn1, pn2)


../results/linearprobe_50epochs_paper_final2/CLIP-convnext_base_w-laion_aesthetic-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/linearprobe_50epochs_paper_final2/CLIP-convnext_base_w-laion2B-s13B-b82K_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/linearprobe_50epochs_paper_final2/deit_small_patch16_224.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/linearprobe_50epochs_paper_final2/robust_resnet50_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/linearprobe_50epochs_paper_final2/vit_small_patch16_224.augreg_in21k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/linearprobe_50epochs_paper_final2/convnext_base.fb_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/linearprobe_50epochs_paper_final2/resnet50.a1_in1k_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/linearprobe_50epochs_paper_final2/robust_vit_base_patch16_224_uc-merced-land-use-dataset_TRADES_v2.pkl
../results/linearprobe_50epochs_paper_final2/vit_base_patch16_224.mae_uc-merc

In [9]:
import pandas as pd
import yaml

df = pd.DataFrame(final_data)
df = df[df["model_size"].isin(['medium', 'large'])]
# df.to_csv("./{}.csv".format("result_with_missing_values_max"))

def get_restart_job(row):
    is_missing = lambda col: pd.isna(row[col])

    missing = {col: is_missing(col) for col in ['clean_acc', 'Linf_acc', 'L1_acc', 'L2_acc', 'common_acc']}

    if all(missing.values()):
        return "job1_hpo.sh"
    elif not missing['clean_acc'] and not missing['Linf_acc'] and missing['L1_acc']:
        return "job4_test_l1.sh"
    elif not missing['clean_acc'] and not missing['Linf_acc'] and not missing['L1_acc'] and missing['L2_acc']:
        return "job5_test_l2.sh"
    elif not missing['clean_acc'] and not missing['Linf_acc'] and not missing['L1_acc'] and not missing['L2_acc'] and missing['common_acc']:
        return "job6_test_common.sh"
    else:
        return None  # means no job needs to be restarted

df['restart_from'] = df.apply(get_restart_job, axis=1)

to_restart = df[df['restart_from'].notna()][['backbone', 'dataset', 'loss_function', 'restart_from']]
to_restart = to_restart[ to_restart.restart_from == "job1_hpo.sh" ]
to_restart.to_csv("./to_relaunch_jonas.csv")
to_restart


,backbone,dataset,loss_function,restart_from
14,vit_base_patch16_clip_224.laion2b,uc-merced-land-use-dataset,TRADES_v2,job1_hpo.sh
16,robust_convnext_tiny,uc-merced-land-use-dataset,TRADES_v2,job1_hpo.sh
30,coatnet_0_rw_224.sw_in1k,uc-merced-land-use-dataset,TRADES_v2,job1_hpo.sh
31,coatnet_2_rw_224.sw_in12k_ft_in1k,uc-merced-land-use-dataset,TRADES_v2,job1_hpo.sh
32,coatnet_2_rw_224.sw_in12k,uc-merced-land-use-dataset,TRADES_v2,job1_hpo.sh
70,coatnet_0_rw_224.sw_in1k,stanford_cars,TRADES_v2,job1_hpo.sh
71,coatnet_2_rw_224.sw_in12k_ft_in1k,stanford_cars,TRADES_v2,job1_hpo.sh
72,coatnet_2_rw_224.sw_in12k,stanford_cars,TRADES_v2,job1_hpo.sh
104,eva02_tiny_patch14_224.mim_in22k,caltech101,TRADES_v2,job1_hpo.sh
110,coatnet_0_rw_224.sw_in1k,caltech101,TRADES_v2,job1_hpo.sh


In [38]:

from matplotlib import cm
from matplotlib.colors import to_hex
import re


import pandas as pd
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
from process_database import process_grouped_df, process_rankings, compute_odds_ratio_by_group

import pandas as pd

# Your model type mapping
model_type_map = {
    'convnext_base': "fully convolutional",
    'convnext_tiny': "fully convolutional",
    'deit_small': "fully attention",
    'vit_base': "fully attention",
    'vit_small': "fully attention",
    'resnet50': "fully convolutional",
    'eva02_base': "fully attention",
    'eva02_tiny': "fully attention",
    'swin_base': "fully attention",
    'swin_tiny': "fully attention",
    'coatnet_0': "hybrid",
    'coatnet_2': "hybrid",
    'regnetx_004': "fully convolutional",
    'efficientnet-b0': "fully convolutional", 
    'deit_tiny': "fully attention",
    'mobilevit-small': "hybrid",
    'mobilenetv3': "fully convolutional",
    'edgenext_small': "fully convolutional",
    'coat_tiny': "hybrid",
}

# Colormap for each model type
type_to_cmap = {
    'fully convolutional': cm.Purples,     # deep violet → lavender
    'fully attention': cm.Oranges,         # strong orange → light peach
    'hybrid': cm.Greens                    # forest → mint
}

# Extract base backbone name from full backbone string
def extract_base_name(backbone_name):
    for base in model_type_map:
        if base in backbone_name:
            return base
    return 'unknown'


# Define line styles per loss function
line_styles = {
    'CLASSIC_AT': 'solid',
    'TRADES_v2': 'dot'
}


# Define mapping for renaming
metric_rename_map = {
    'sum_L1_acc': "T(1)",
    'sum_L2_acc': "T(2)",
    'sum_Linf_acc': "T(∞)",
    'sum_clean_acc': "T(∅)",
    'sum_common_acc':"T(cmn)"
}

# Apply the mapping to metrics list and add loop closing term
theta_labels = [metric_rename_map[m] for m in metric_rename_map] + [metric_rename_map[list(metric_rename_map.keys())[0]]]


from plotly.colors import qualitative
color_palette = qualitative.Bold

from process_database import process_grouped_df, process_rankings, compute_odds_ratio_by_group

for size_id,size in [ (0,'small'), (1,'medium'), (2,'large') ]: 
    grouped_df = process_grouped_df(final_data, size)
    grouped_df = process_rankings(grouped_df)

    df1 = grouped_df.copy()

    metrics_to_sum = ["L1_acc", "L2_acc", "Linf_acc", "clean_acc", "common_acc"]

    for metric in metrics_to_sum:
        metric_cols = [col for col in df1.columns if col[1] == metric]
        df1[f'sum_{metric}'] = df1[metric_cols].sum(axis=1)

    df_per_row_sums = df1[[f'sum_{m}' for m in metrics_to_sum]]

    df_per_row_sums.columns = df_per_row_sums.columns.get_level_values(0)

    df_per_row_sums.columns.name = 'metric'

    df_per_row_sums

    # Get all unique backbones
    backbones = sorted(set(backbone for backbone, _,_,_ in df_per_row_sums.index))

    # Map backbones to model type
    backbone_model_type = {b: model_type_map.get(extract_base_name(b), 'unknown') for b in backbones}

    # Assign gradient color per backbone based on model type
    color_map = {}
    for model_type in ['fully convolutional', 'fully attention', 'hybrid']:
        bks = [bk for bk in backbones if backbone_model_type[bk] == model_type]
        n = len(bks)
        for i, bk in enumerate(bks):
            start, end = 0.5, 0.9  # avoid very light or very dark edges
            position = start + (end - start) * (i / max(n - 1, 1))
            color = to_hex(type_to_cmap[model_type](position))
            color_map[bk] = color


    # Define metrics
    metrics = ['sum_L1_acc', 'sum_L2_acc', 'sum_Linf_acc', 'sum_clean_acc', 'sum_common_acc']

    # Get unique backbones
    unique_backbones = sorted( set(backbone for backbone, _,_,_ in df_per_row_sums.index) )

    fig = go.Figure()

    for (backbone, loss, _, _), row in df_per_row_sums.iterrows():
        fig.add_trace(go.Scatterpolar(
            r=row.tolist() + [row.tolist()[0]],  # Close the loop
            theta=theta_labels,
            mode='lines',
            name=f"{backbone} ({loss})",
            line=dict(
                width=2,
                color=color_map[backbone],
                dash=line_styles.get(loss, 'solid')
            )
        ))

    fig.update_layout(
        margin=dict(l=40, r=25, t=25, b=25),  # remove all outer margins
        width=400,    # in pixels (~2.5 inches at 96 DPI)
        height=400,    # in pixels (~1.875 inches)
        paper_bgcolor='white',
        plot_bgcolor='white',

        polar=dict(
            radialaxis=dict(
                visible=True,
                range=[0, 4.5],
                showline=True,
                linewidth=1,
                linecolor="lightgrey",
                gridcolor="lightgrey",
                gridwidth=0.5,
            ),
            angularaxis=dict(
                direction="clockwise",
                showline=True,
                linewidth=1,
                linecolor="lightgrey",
                gridcolor="lightgrey",
                gridwidth=0.5,
            ),
            bgcolor='white'
        ),
        legend=dict(
            x=0,
            y=1.0,
            xanchor='left',
            yanchor='top',
            bgcolor='rgba(255,255,255,0.9)',
            bordercolor='lightgrey',
            borderwidth=1,
            font=dict(size=11),
        ),
        showlegend=False,
        font=dict(color='black'),
    )

    fig.show()

    fig.write_image("./paper_figures/radar_plot_{}_{}.png".format(size, pn2), scale=3  )                # upscale for higher DPI (1 = default)

Percentage of NaN values: 4.66%


Percentage of NaN values: 4.66%


Percentage of NaN values: 4.66%
